Currently, this notebook must be run in a NeMo container.
An example command to launch the container:
```bash
docker run --gpus all -it --rm -v <your_nemo_dir>:/NeMo --shm-size=8g -p 8888:8888 -p 6006:6006 --ulimit memlock=-1 --ulimit stack=67108864 <your_nemo_container>
```

In [1]:
# Update megatron version to the newest.
!cd /workspace && python -m pip install -e git+https://github.com/NVIDIA/Megatron-LM#egg=megatron-core

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining megatron-core from git+https://github.com/NVIDIA/Megatron-LM#egg=megatron-core
  Cloning https://github.com/NVIDIA/Megatron-LM to ./src/megatron-core
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/Megatron-LM /workspace/src/megatron-core
  Resolved https://github.com/NVIDIA/Megatron-LM to commit 53a350eddbce38f036fe5884c3fec955cf710b9e
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for megatron-core (pyproject.toml) ... done
  Created wheel for megatron-core: filename=megatron_core-0.5.0rc0-0.editable-cp310-cp310-linux_x86_64.whl size=8667 sha256=a44e552c0568abfec018644abd88

In [2]:
%cd /NeMo/tutorials/nlp
BRANCH='main'
import os
import wget
import sys
sys.path.insert(0, "../..")  # find the local nemo first before the installed nemo

[Errno 2] No such file or directory: '/NeMo/tutorials/nlp'
/mnt/code


In [24]:
BRANCH='main'
import os
import wget
import sys
sys.path.insert(0, "/workspace/nemo/")  # find the local nemo first before the installed nemo

### Introduction

This notebook demonstrates how to apply PEFT in NeMo. For brevity, we have chosen LoRA as the PEFT technique and GPT as the language model, but the same recipe can be used for other PEFT techniques and language models, as described in the [Training](#training) section.

 The implementation of LoRA is based on the paper, [LoRA: Low-Rank Adaptation of Large Language Models](https://openreview.net/pdf?id=nZeVKeeFYf9) by Hu et al.

This example demonstrates how to:

    1. Train a LoRA model on a simple Extractive QA task.
    2. Inspect the trained LoRA model showing the parameters it contains.
    3. Run inference with the base model with the LoRA parameters.

### Tasks and Datasets
We will be using LoRA to teach our GPT model to do Extractive Question Answering.

We will be using the [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) reading comprehension dataset, consisting of questions posed by crowd workers on a set of Wikipedia articles, where the answer to every question is a segment of text. More information on [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) can be found on their website or in their paper by Rajpurkar et. al "[Know What You Don’t Know: Unanswerable Questions for SQuAD](https://arxiv.org/pdf/1806.03822.pdf)".

LoRA (and all PEFT tuning) models expect at least two fields in the jsonl files. The `input` field should contain all the tokens necessary for the model to generate the `output`. For example for extractive QA, the `input` should contain the context text as well as the question.

```
[
    {"input": "User: Context: [CONTEXT_1] Question: [QUESTION_1]\n\nAssistant:", "output": [ANSWER_1]},
    {"input": "User: Context: [CONTEXT_2] Question: [QUESTION_2]\n\nAssistant:", "output": [ANSWER_2]},
    {"input": "User: Context: [CONTEXT_3] Question: [QUESTION_3]\n\nAssistant:", "output": [ANSWER_3]},
]
```
Note that we use keywords in the input like `Context:`, `Question:` to separate the text representing the context and question. We also use the keyword `User:` and end each of the input with `\n\nAssistant:` tokens. These are recommended because NeMo's instruction-tuned models are trained with a prefix of `User:` and suffix `\n\nAssistant:`.

In [25]:
# You can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "data"
NEMO_DIR = "."
os.makedirs(DATA_DIR, exist_ok=True)
SQUAD_DIR = os.path.join(DATA_DIR, "SQuAD")
os.makedirs(SQUAD_DIR, exist_ok=True)


For each dataset we have preprocessing scripts pre-written in NeMo's example directory located in `examples/nlp`. Let's download those now. 

In [26]:
# download the preprocessing scripts from github for the purpose of this tutorial
! wget -nc https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/dataset_processing/nlp/squad/prompt_learning_squad_preprocessing.py

File ‘prompt_learning_squad_preprocessing.py’ already there; not retrieving.



Now let's down load and process the dataset.

In [27]:
# Download the SQuAD dataset
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
!mv train-v1.1.json {SQUAD_DIR}
!mv dev-v1.1.json {SQUAD_DIR}

--2024-03-06 00:19:55--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘train-v1.1.json’

train-v1.1.json     100%[===================>]  28.88M   121MB/s    in 0.2s    

2024-03-06 00:19:55 (121 MB/s) - ‘train-v1.1.json’ saved [30288272/30288272]

--2024-03-06 00:19:55--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json’

dev-v1.1.json       1

In [28]:
# Preprocess squad data
!python prompt_learning_squad_preprocessing.py --sft-format --data-dir {SQUAD_DIR}

Saving train split to data/SQuAD/squad_train.jsonl
100%|█████████████████████████████████| 87599/87599 [00:00<00:00, 145821.80it/s]
Saving val split to data/SQuAD/squad_val.jsonl
100%|█████████████████████████████████| 10570/10570 [00:00<00:00, 140944.89it/s]
Saving test split to data/SQuAD/squad_test_ground_truth.jsonl
100%|█████████████████████████████████| 10570/10570 [00:00<00:00, 131138.30it/s]
Saving test split to data/SQuAD/squad_test.jsonl
100%|█████████████████████████████████| 10570/10570 [00:00<00:00, 141192.22it/s]


In [29]:
# What the squad dataset looks like after processing
! head -200 $SQUAD_DIR/squad_train.jsonl > $SQUAD_DIR/squad_short_train.jsonl
! head -20 $SQUAD_DIR/squad_val.jsonl > $SQUAD_DIR/squad_short_val.jsonl
! head -4 $SQUAD_DIR/squad_short_val.jsonl
! head -4 $SQUAD_DIR/squad_short_train.jsonl

{"input": "User: Context:Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24\u201310 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the \"golden anniversary\" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as \"Super Bowl L\"), so that the logo could prominently feature the Arabic numerals 50. Question:Which NFL team represented the AFC at Super Bowl 50?\n\nAssistant:", "output": "Denver Broncos"}
{"input": "User: Context:Super Bowl 50 was an American football game to determine th

### Model Config Setup
Now we will begin setting up the config file needed for PEFT tuning. We use a single config for all supported PEFT methods (LoRA, Adapter, IA3 and P-Tuning, as well as combinations of these).  All PEFT methods use the GPT finetuning class `MegatronGPTSFTModel` as the frozen base network, and use the `add_adapter()` method to add adapter weights for PEFT.

Let's create a config object for LoRA training.

In [30]:
from omegaconf import OmegaConf

CONFIG_DIR = os.path.join(NEMO_DIR, "conf")
os.makedirs(CONFIG_DIR, exist_ok=True)

# Download the example config file
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/tuning/conf/megatron_gpt_finetuning_config.yaml', CONFIG_DIR)

# Load the example config file so we can start editing it
CONFIG_PATH = os.path.join(CONFIG_DIR, "megatron_gpt_finetuning_config.yaml")
config = OmegaConf.load(CONFIG_PATH)

The `config` contains several attributes required by the `MegatronGPTSFTModel`. First we will set the training data path and the validation data path in the config.
The `config` allows us to set a list of `jsonl` files as training files and sample examples from each file with different probabilities. For simplicity, we are going to use just one training file and thus the sampling probability is set to `1.0`

We can also monitor validation loss from multiple validation files during training. Again for simplicity we will use just one validation file.

In [31]:
config.model.data.train_ds.file_names = [f"{SQUAD_DIR}/squad_short_train.jsonl"]
config.model.data.train_ds.concat_sampling_probabilities=[1.0]
config.model.data.validation_ds.file_names = [f"{SQUAD_DIR}/squad_short_val.jsonl"]
config.model.data.validation_ds.names=["squad_val"]

### PEFT Config
The attribute [config.model.peft](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/tuning/conf/megatron_gpt_finetuning_config.yaml#L78) contains settings that control the PEFT training method and its related hyperpameters. We currently support `lora`, `adapter`, `ptuning` and `ia3`. We can instruct the training script to use one of these methods by setting the config.model.peft.peft_scheme attribute.

The other hyperparams associated with lora tuning are present in the [config.model.peft.lora_tuning](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/tuning/conf/megatron_gpt_finetuning_config.yaml#L92) attribute.

In [32]:
config.model.peft.peft_scheme="lora"  # we can also set this to adapter or ptuning or ia3
print(OmegaConf.to_yaml(config.model.peft.lora_tuning))

target_modules:
- attention_qkv
adapter_dim: 32
alpha: ${model.peft.lora_tuning.adapter_dim}
adapter_dropout: 0.0
column_init_method: xavier
row_init_method: zero
layer_selection: null
weight_tying: false
position_embedding_strategy: null



**Note:** In the original LoRA paper each attention projection (`K`, `Q`, `V` and `O`) can have their own Low-Rank projections. However, NeMo's attention implementation fuses `KQV` into a single projection and thus our LoRA implementation learns a single Low-Rank projection for `KQV` in a combined fashion. We do not support LoRA for the `O` matrix at this point.

### Prompt Formatting
The `config.model.data.train_ds.prompt_template` attribute allows us to further tweak the format of the input and output if needed. In this example, we have already incorporated our format inside the `jsonl` file during preprocessing, so we can keep the `prompt_template` in the config simple. (See previous section on Data Preparation).

In [33]:
config.model.data.train_ds.prompt_template ="{input} {output}"

### Setting the Pretrained GPT Model
Next we will set the "base language model" upon which we will perform LoRA tuning. Obviously, larger base models will have better performance on downstream tasks but for the purposes of this tutorial we will use a small 345M parameter GPT model.

In [34]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
megatron_gpt_345m_nemo_url = MegatronGPTModel.list_available_models()[0].location
megatron_gpt_345m_nemo_url # should point to the 345m megatron gpt model '.nemo' file

'https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo'

If we wanted to use the GPT model class directly, we could instantiate a trainer then download the model by calling running 
`gpt_model = MegatronGPTModel.from_pretrained(model_name="megatron_gpt_345m", trainer=trainer).cuda()`. But we just need the `.nemo` file in our working NeMo directory in this tutorial, so we will download it using `wget`. 

In [35]:
# Download the model from NGC
gpt_file_name = "megatron_gpt_345m.nemo"

In [36]:
!wget  -nc --content-disposition {megatron_gpt_345m_nemo_url} -O {NEMO_DIR}/{gpt_file_name}

File ‘./megatron_gpt_345m.nemo’ already there; not retrieving.


Now that we have a `.nemo` GPT file to work with. We need to add its path in our prompt learning config. 

In [37]:
# Set GPT model path on prompt learning config
config.model.restore_from_path = gpt_file_name

Next, we will set where we want to save all the intermediate training logs and checkpoints. As well as other training settings such as: number of training steps, batch size and validation check interval, and num_workers for data processing.

In [38]:
config.exp_manager.exp_dir=f"{NEMO_DIR}/peft_lora"
config.exp_manager.explicit_log_dir="training_info"
config.trainer.max_steps=100
config.model.micro_batch_size=1
config.model.global_batch_size=4
config.trainer.val_check_interval=50
config.model.data.train_ds.num_workers=0  # 0 is recommended which just uses the main thread to process training examples
config.model.data.validation_ds.num_workers=0 # 0 is recommended which just uses the main thread to process the validation examples

Let's have a look at all the values we've set in the model config. You can change any of these values in the same manner we've been using above. 

In [39]:
# Final model config
print(OmegaConf.to_yaml(config.model))

seed: 1234
tensor_model_parallel_size: 1
pipeline_model_parallel_size: 1
global_batch_size: 4
micro_batch_size: 1
restore_from_path: megatron_gpt_345m.nemo
resume_from_checkpoint: null
save_nemo_on_validation_end: false
sync_batch_comm: false
megatron_amp_O2: false
sequence_parallel: false
activations_checkpoint_granularity: null
activations_checkpoint_method: null
activations_checkpoint_num_layers: null
activations_checkpoint_layers_per_pipeline: null
answer_only_loss: true
gradient_as_bucket_view: false
hidden_dropout: 0.0
attention_dropout: 0.0
ffn_dropout: 0.0
fsdp: false
fsdp_sharding_strategy: full
fsdp_grad_reduce_dtype: fp32
fsdp_sharded_checkpoint: false
fsdp_use_orig_params: false
peft:
  peft_scheme: lora
  restore_from_path: null
  adapter_tuning:
    type: parallel_adapter
    adapter_dim: 32
    adapter_dropout: 0.0
    norm_position: pre
    column_init_method: xavier
    row_init_method: zero
    norm_type: mixedfusedlayernorm
    layer_selection: null
    weight_tying:

### Building the PyTorch Lightning Trainer
NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [40]:
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPStrategy
import torch
import pytorch_lightning as pl
from nemo.collections.nlp.parts.megatron_trainer_builder import MegatronTrainerBuilder

# let's modify some trainer configs
# check if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.accelerator = accelerator
config.trainer.devices = 1
config.trainer.max_epochs = 4
config.trainer.val_check_interval = 1.0

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# setup cluster environment parameters"
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'

trainer = MegatronTrainerBuilder(config).create_trainer()

print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

[NeMo I 2024-03-06 00:20:00 megatron_trainer_builder:49] Detected interactive environment, using NLPDDPStrategyNotebook


[NeMo W 2024-03-06 00:20:00 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
      rank_zero_warn(
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


Trainer config - 

devices: 1
accelerator: gpu
num_nodes: 1
precision: 16
logger: false
enable_checkpointing: false
use_distributed_sampler: false
max_epochs: 4
max_steps: 100
log_every_n_steps: 10
val_check_interval: 1.0
gradient_clip_val: 1.0



In [41]:
print(OmegaConf.to_yaml(config.exp_manager))

explicit_log_dir: training_info
exp_dir: ./peft_lora
name: ${name}
create_wandb_logger: false
wandb_logger_kwargs:
  project: null
  name: null
resume_if_exists: true
resume_ignore_no_checkpoint: true
create_checkpoint_callback: true
checkpoint_callback_params:
  monitor: validation_${model.data.validation_ds.metric.name}
  save_top_k: 1
  mode: min
  save_nemo_on_train_end: true
  filename: ${name}--{${exp_manager.checkpoint_callback_params.monitor}:.3f}-{step}-{consumed_samples}
  model_parallel_size: ${model.tensor_model_parallel_size}
  always_save_nemo: false
  save_best_model: true
create_early_stopping_callback: true
early_stopping_callback_params:
  monitor: val_loss
  mode: min
  min_delta: 0.001
  patience: 10
  verbose: true
  strict: false



### Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [42]:
from nemo.utils.exp_manager import exp_manager

# Set name of the experiment 
config.name = 'lora_example_tuning'
config.exp_manager.resume_if_exists = False

# Init the experiment manager and view the exp_dir
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
exp_dir = str(exp_dir)
print(exp_dir)

[NeMo E 2024-03-06 00:20:01 exp_manager:684] exp_manager received explicit_log_dir: training_info and at least one of exp_dir: ./peft_lora, or version: None. Please note that exp_dir, name, and version will be ignored.
[NeMo W 2024-03-06 00:20:01 exp_manager:689] Exp_manager is logging to training_info, but it already exists.


[NeMo I 2024-03-06 00:20:01 exp_manager:394] Experiments will be logged at training_info
[NeMo I 2024-03-06 00:20:01 exp_manager:835] TensorboardLogger has been set up


[NeMo W 2024-03-06 00:20:01 exp_manager:931] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 100. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


training_info


### Training
We now set up the process for training a LoRA model. We first require a config that contains details about the base language model upon which we will train our LoRA model. So we first extract the `model_cfg` from the checkpoint and update it with any new settings we employ in our current (LoRA) `config`. These are combined in the `merge_cfg_with` function.



In [43]:
from nemo.collections.nlp.models.language_modeling.megatron_gpt_sft_model import MegatronGPTSFTModel

model_cfg = MegatronGPTSFTModel.merge_cfg_with(config.model.restore_from_path, config)

Next, we instantiate the GPT model class and add the LoRA adapter
When we call `add_adapter`, the model prints out the parameter count before and after the operation. We can clearly see the number of trainable parameters increase after adding the adapter.
To print the parameter count manually, we can call `model.summarize()`.

In [44]:
from nemo.collections.nlp.parts.peft_config import LoraPEFTConfig

model = MegatronGPTSFTModel.restore_from(config.model.restore_from_path, model_cfg, trainer=trainer)
model.add_adapter(LoraPEFTConfig(model_cfg))
# print("Parameter count manually:\n", model.summarize())

[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: virtual_pipeline_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: gradient_accumulation_fusion in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field

[NeMo I 2024-03-06 00:20:05 megatron_init:234] Rank 0 has data parallel group: [0]
[NeMo I 2024-03-06 00:20:05 megatron_init:237] All data parallel group ranks: [[0]]
[NeMo I 2024-03-06 00:20:05 megatron_init:238] Ranks 0 has data parallel rank: 0
[NeMo I 2024-03-06 00:20:05 megatron_init:246] Rank 0 has model parallel group: [0]
[NeMo I 2024-03-06 00:20:05 megatron_init:247] All model parallel group ranks: [[0]]
[NeMo I 2024-03-06 00:20:05 megatron_init:257] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-03-06 00:20:05 megatron_init:261] All tensor model parallel group ranks: [[0]]
[NeMo I 2024-03-06 00:20:05 megatron_init:262] Rank 0 has tensor model parallel rank: 0
[NeMo I 2024-03-06 00:20:05 megatron_init:276] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2024-03-06 00:20:05 megatron_init:288] Rank 0 has embedding group: [0]
[NeMo I 2024-03-06 00:20:05 megatron_init:294] All pipeline model parallel group ranks: [[0]]
[NeMo I 2024-03-06 00:20:05 megatron_init:295]

[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: virtual_pipeline_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: gradient_accumulation_fusion in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field

[NeMo I 2024-03-06 00:20:05 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-gpt-345m, custom vocab file: /tmp/tmpwov5mnc3/bfcdca5e44814366bdb5dcd651325152_gpt2-vocab.json, and merges file: /tmp/tmpwov5mnc3/315a11fd68be49d6abdb34363e8c4997_gpt2-merge.txt
[NeMo I 2024-03-06 00:20:05 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: gpt2, vocab_file: /tmp/tmpwov5mnc3/bfcdca5e44814366bdb5dcd651325152_gpt2-vocab.json, merges_files: /tmp/tmpwov5mnc3/315a11fd68be49d6abdb34363e8c4997_gpt2-merge.txt, special_tokens_dict: {}, and use_fast: False
[NeMo I 2024-03-06 00:20:05 megatron_base_model:315] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.


[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: virtual_pipeline_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field.name: gradient_accumulation_fusion in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-03-06 00:20:05 megatron_base_model:821] The model: MegatronGPTSFTModel() does not have field

[NeMo I 2024-03-06 00:20:05 build_model:143]  > number of parameters on (tensor, pipeline) model parallel rank (0, 0): 354871296
[NeMo I 2024-03-06 00:20:06 nlp_overrides:752] Model MegatronGPTSFTModel was successfully restored from /mnt/code/megatron_gpt_345m.nemo.
[NeMo I 2024-03-06 00:20:06 nlp_adapter_mixins:182] Before adding PEFT params:
      | Name  | Type     | Params
    -----------------------------------
    0 | model | GPTModel | 354 M 
    -----------------------------------
    0         Trainable params
    354 M     Non-trainable params
    354 M     Total params
    1,419.485 Total estimated model params size (MB)
[NeMo I 2024-03-06 00:20:08 nlp_adapter_mixins:195] After adding PEFT params:
      | Name  | Type     | Params
    -----------------------------------
    0 | model | GPTModel | 358 M 
    -----------------------------------
    3.1 M     Trainable params
    354 M     Non-trainable params
    358 M     Total params
    1,432.068 Total estimated model param

Simply substitute with the `MegatronT5SFTModel` class to use T5 instead of GPT.

To use a different PEFT method, you can use a different config class in place of `LoraPEFTConfig`, such as `CanonicalAdaptersPEFTConfig`, `IA3PEFTConfig`, `PtuningPEFTConfig`. You can also use a combination of the methods by passing in a list:
`model.add_adapter([LoraPEFTConfig(model_cfg), PtuningPEFTConfig(model_cfg)])`

We're now ready to start training.

In [45]:
trainer.fit(model)

[NeMo W 2024-03-06 00:20:08 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:153: UserWarning: The `batch_idx` argument in `MegatronGPTSFTModel.on_train_batch_start` hook may not match with the actual batch index when using a `dataloader_iter` argument in your `training_step`.
      rank_zero_warn(
    
[NeMo W 2024-03-06 00:20:08 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:153: UserWarning: The `batch_idx` argument in `MegatronGPTSFTModel.on_train_batch_end` hook may not match with the actual batch index when using a `dataloader_iter` argument in your `training_step`.
      rank_zero_warn(
    


[NeMo I 2024-03-06 00:20:08 megatron_gpt_sft_model:752] Building GPT SFT validation datasets.


[NeMo W 2024-03-06 00:20:08 megatron_gpt_sft_model:258] Set dataset max_seq_length to max_position_embeddings 1024 if using learned_absolute position embedding


[NeMo I 2024-03-06 00:20:08 text_memmap_dataset:116] Building data files
[NeMo I 2024-03-06 00:20:08 text_memmap_dataset:525] Processing 1 data files using 2 workers
[NeMo I 2024-03-06 00:20:08 text_memmap_dataset:535] Time building 0 / 1 mem-mapped files: 0:00:00.231390
[NeMo I 2024-03-06 00:20:08 text_memmap_dataset:525] Processing 1 data files using 2 workers
[NeMo I 2024-03-06 00:20:08 text_memmap_dataset:535] Time building 0 / 1 mem-mapped files: 0:00:00.232171
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:158] Loading data files
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:249] Loading data/SQuAD/squad_short_val.jsonl
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:161] Time loading 1 mem-mapped files: 0:00:00.002020
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:165] Computing global indices
[NeMo I 2024-03-06 00:20:09 megatron_gpt_sft_model:755] Length of val dataset: 20
[NeMo I 2024-03-06 00:20:09 megatron_gpt_sft_model:766] Building GPT SFT traing datasets.


[NeMo W 2024-03-06 00:20:09 megatron_gpt_sft_model:258] Set dataset max_seq_length to max_position_embeddings 1024 if using learned_absolute position embedding


[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:116] Building data files
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:525] Processing 1 data files using 2 workers
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:535] Time building 0 / 1 mem-mapped files: 0:00:00.236648
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:525] Processing 1 data files using 2 workers
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:535] Time building 0 / 1 mem-mapped files: 0:00:00.236274
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:158] Loading data files
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:249] Loading data/SQuAD/squad_short_train.jsonl
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:161] Time loading 1 mem-mapped files: 0:00:00.001549
[NeMo I 2024-03-06 00:20:09 text_memmap_dataset:165] Computing global indices
 > WARNING: could not find index map file data/SQuAD/squad_short_train.jsonl_squad_short_train.jsonl_indexmap_402mns_1022msl_0.00ssp_1234s.npy, building the indices on rank 0 ...
[Ne

/usr/bin/ld: cannot open output file helpers.cpython-310-x86_64-linux-gnu.so: Permission denied
collect2: error: ld returned 1 exit status
make: *** [Makefile:23: helpers.cpython-310-x86_64-linux-gnu.so] Error 1
[NeMo E 2024-03-06 00:20:14 dataset_utils:84] Making C++ dataset helpers module failed, exiting.


make: Leaving directory '/usr/local/lib/python3.10/dist-packages/nemo/collections/nlp/data/language_modeling/megatron'


SystemExit: 1

[NeMo W 2024-03-06 00:20:15 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
      warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
    


Once training is completed you should see a saved '.nemo' file in this folder `{config.exp_manager.explicit_log_dir}/checkpoints`. This checkpoint will only contain the trained adapter weights, and not the frozen base model weights.

In [28]:
# The trained '.nemo' model is saved in the location below:
! ls -lh {config.exp_manager.explicit_log_dir}/checkpoints
print(config.exp_manager.explicit_log_dir)

ls: cannot access 'training_info/checkpoints': No such file or directory
training_info


### Inference
The model object from `trainer.fit(model)` is also capable of doing inference. For the tutorial, however, we will re-load the saved `.nemo` lora model along with a `.nemo` base language model to simulate a more realistic scenario (where training does not happen right before inference).

Run the cell below to reimport libraries and classes in case you did not run the training cells above.

In [29]:
# reimport libraries and classes in case one wants to only run cells from the Inference section
%cd /NeMo/tutorials/nlp
import wget, os, sys
sys.path.insert(0, "../..")  # find the local nemo first before the installed nemo
from omegaconf import OmegaConf
from nemo.collections.nlp.parts.megatron_trainer_builder import MegatronTrainerBuilder
from nemo.collections.nlp.parts.peft_config import LoraPEFTConfig
from nemo.collections.nlp.models.language_modeling.megatron_gpt_sft_model import MegatronGPTSFTModel

NEMO_DIR = "."
DATA_DIR = "data"
CONFIG_DIR = os.path.join(NEMO_DIR, "conf")
SQUAD_DIR = os.path.join(DATA_DIR, "SQuAD")


[Errno 2] No such file or directory: '/NeMo/tutorials/nlp'
/mnt/code


First, we will load and modify a config file that will be used for inference.


In [30]:
# Download the example config file
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/tuning/conf/megatron_gpt_generate_config.yaml', CONFIG_DIR)

'./conf/megatron_gpt_generate_config.yaml'

In [31]:
# Load the example config file so we can start editing it
CONFIG_EVAL_PATH = os.path.join(CONFIG_DIR, "megatron_gpt_generate_config.yaml")
config_eval = OmegaConf.load(CONFIG_EVAL_PATH)

We are going to modify the `config_eval` object that we created above. We will set the base language model as the `345m` model we downloaded earlier.

Additionally, we will also set the `model.peft.restore_from_path` with the lora model we just trained. For the tutorial we will just use the validation data for inference as well.

In [32]:
config_eval.model.restore_from_path="megatron_gpt_345m.nemo"
config_eval.model.peft.restore_from_path="./training_info/checkpoints/lora_example_tuning.nemo"
config_eval.model.data.test_ds.file_names=[f"{SQUAD_DIR}/squad_short_val.jsonl"]
config_eval.model.data.test_ds.names=["test_set"]
config_eval.model.data.test_ds.global_batch_size=1
config_eval.model.data.test_ds.micro_batch_size=1
config_eval.model.data.test_ds.tokens_to_generate=30
config_eval.inference.greedy=True

In [33]:
trainer_eval = MegatronTrainerBuilder(config_eval).create_trainer()

[NeMo I 2024-03-05 01:17:45 megatron_trainer_builder:49] Detected interactive environment, using NLPDDPStrategyNotebook


[NeMo W 2024-03-05 01:17:45 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
      rank_zero_warn(
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


The `config_eval` object is the hydra config at "inference/test time". This means it should contain information relevant for inference/test time, although some properties that were set at training time are still relevant. For example, whether training was done with `BOS` enabled or not, and other model specific attributes.

So we extract the relevant information from the '.nemo' file of the lora model we just trained using the `merge_inference_cfg` function.

In [34]:
eval_model_cfg = MegatronGPTSFTModel.merge_inference_cfg(config_eval.model.peft.restore_from_path, config_eval)

FileNotFoundError: Can't find /mnt/code/training_info/checkpoints/lora_example_tuning.nemo

The cell below is required if you are running the notebook end-to-end, and if you use a different batch size for training and evaluation. In this case, the microbatch calculator needs to be rest. If you are running training only or inference only, feel free to ignore this cell.

In [35]:
from apex.transformer.pipeline_parallel.utils import _reconfigure_microbatch_calculator
_reconfigure_microbatch_calculator(
    rank=0,
    rampup_batch_size=None,
    global_batch_size=config_eval.model.global_batch_size,
    micro_batch_size=config_eval.model.micro_batch_size,
    data_parallel_size=1,
)

[NeMo W 2024-03-05 01:17:50 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/apex/transformer/pipeline_parallel/utils.py:81: UserWarning: This function is only for unittest
      warnings.warn("This function is only for unittest")
    


Then, we load the base language model as well as the lora model we just trained.

In [36]:
model_eval = MegatronGPTSFTModel.restore_from(config_eval.model.restore_from_path, eval_model_cfg, trainer=trainer_eval)
model_eval.load_adapters(config_eval.model.peft.restore_from_path, LoraPEFTConfig(eval_model_cfg))
model_eval.freeze()

print("Parameter count manually:\n", model_eval.summarize())

NameError: name 'eval_model_cfg' is not defined

Next, we prepare the dataset and the dataloader objects that the model will perform inference on.

In [ ]:
_test_ds = model_eval._build_dataset(eval_model_cfg.data.test_ds, is_train=False)
from torch.utils.data import DataLoader
request_dl = DataLoader(
    dataset=_test_ds[0],
    batch_size=eval_model_cfg.data.test_ds.global_batch_size,
    collate_fn=_test_ds[0].collate_fn,
)
config_inference = OmegaConf.to_container(config_eval.inference, resolve=True)
model_eval.set_inference_config(config_inference)

And finally, we call `trainer.predict` which triggers the inference process. The `response` object contains the outputs of the model.

In [ ]:
response = trainer_eval.predict(model_eval, request_dl)
for batch in response:
    for s in batch['sentences']:
        print(f"{s}\n\n")